In [1]:
!pip install tensorflow==2.0.0-beta1

In [2]:
from sklearn import datasets
from sklearn.utils import check_array
iris = datasets.load_iris()

In [37]:
X = iris.data  # we only take the first two features.
Y = iris.target

X = check_array(X, dtype=np.float64,
                ensure_min_samples=2)
Y = check_array(Y, dtype=np.float64,ensure_2d=False)

if Y.ndim == 1:
            Y = Y.reshape(-1, 1)

In [38]:
import tensorflow as tf
tf.__version__

'2.0.0-beta1'

In [107]:
def _nipals_twoblocks_inner_loop(X, Y, mode="A", max_iter=500, tol=1e-06,
                                 norm_y_weights=False):
    """Inner loop of the iterative NIPALS algorithm.
    Provides an alternative to the svd(X'Y); returns the first left and right
    singular vectors of X'Y.  See PLS for the meaning of the parameters.  It is
    similar to the Power method for determining the eigenvectors and
    eigenvalues of a X'Y.
    """
    y_score = Y[:, [0]]
    x_weights_old = 0
    ite = 1
    X_pinv = Y_pinv = None
    eps = 2.220446049250313e-16 #np.finfo(X.dtype).eps
    # Inner loop of the Wold algo.
    while True:
        # 1.1 Update u: the X weights
        if mode == "B":
            if X_pinv is None:
                # We use slower pinv2 (same as np.linalg.pinv) for stability
                # reasons
                X_pinv = pinv2(X, check_finite=False)
            x_weights = np.dot(X_pinv, y_score)
        else:  # mode A
            # Mode A regress each X column on y_score
            x_weights = np.dot(X.T, y_score) / np.dot(y_score.T, y_score)
        
        # If y_score only has zeros x_weights will only have zeros. In
        # this case add an epsilon to converge to a more acceptable
        # solution
        if np.dot(x_weights.T, x_weights) < eps:
            x_weights += eps
        # 1.2 Normalize u
        x_weights /= np.sqrt(np.dot(x_weights.T, x_weights)) + eps
        # 1.3 Update x_score: the X latent scores
        x_score = np.dot(X, x_weights)
        # 2.1 Update y_weights
        if mode == "B":
            if Y_pinv is None:
                Y_pinv = pinv2(Y, check_finite=False)  # compute once pinv(Y)
            y_weights = np.dot(Y_pinv, x_score)
        else:
            # Mode A regress each Y column on x_score
            y_weights = np.dot(Y.T, x_score) / np.dot(x_score.T, x_score)
        # 2.2 Normalize y_weights
        if norm_y_weights:
            y_weights /= np.sqrt(np.dot(y_weights.T, y_weights)) + eps
        # 2.3 Update y_score: the Y latent scores
        y_score = np.dot(Y, y_weights) / (np.dot(y_weights.T, y_weights) + eps)
        # y_score = np.dot(Y, y_weights) / np.dot(y_score.T, y_score) ## BUG
        x_weights_diff = x_weights - x_weights_old
        print(x_weights_diff)
        if np.dot(x_weights_diff.T, x_weights_diff) < tol or Y.shape[1] == 1:
            break
        if ite == max_iter:
            warnings.warn('Maximum number of iterations reached',
                          ConvergenceWarning)
            break
        x_weights_old = x_weights
        ite += 1
    return x_weights, y_weights, ite

In [108]:
_nipals_twoblocks_inner_loop(X,Y)

[[ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 6.86882412]
 [ 6.86882412]
 [ 6.86882412]
 [ 6.86882412]
 [ 6.86882412]
 [ 6.86882412]
 [ 6.86882412]
 [ 6.86882412]
 [ 6.86882412]
 [ 6.86882412]
 [ 6.86882412]
 [ 6.86882412]
 [ 6.86882412]
 [ 6.86882412]
 [ 6.86882412]
 [ 6.86882412]
 [ 6.86882

(array([[0.71916359],
        [0.32804877],
        [0.57813046],
        [0.20236824]]), array([[0.14558533]]), 1)

In [62]:
Xt = tf.convert_to_tensor(X)
Yt = tf.convert_to_tensor(Y)

In [81]:
tf.zeros(X.shape[0])

<tf.Tensor: id=484, shape=(150,), dtype=float32, numpy=
array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)>

In [109]:
Xt.shape

TensorShape([150, 4])

In [116]:
def _nipals_tensorflow(X,Y, max_iter=500, tol=1e-06,norm_y_weights=False):
    y_score = Y
    x_weights_old = 0
    ite = 1
    X_pinv = Y_pinv = None
    eps = 2.220446049250313e-16# np.finfo(X.dtype).eps
    # Inner loop of the Wold algo.
    while True:
        #1. Regress into X
        x_weights = tf.matmul(tf.transpose(X),y_score)/(tf.matmul(tf.transpose(y_score),y_score))
        # 1.2 Normalize u
        x_weights = x_weights/tf.math.sqrt(tf.matmul(tf.transpose(x_weights),x_weights))
        
        # 1.3 Update x_score: the X latent scores
        x_score = tf.matmul(X,x_weights)/tf.matmul(tf.transpose(x_weights),x_weights)
        #2. Regress into Y
        y_weights = tf.matmul(tf.transpose(Y),x_score)/(tf.matmul(tf.transpose(x_score),x_score))
        # 2.1 Normalize y_weights
        y_weights /= y_weights/tf.math.sqrt(tf.matmul(tf.transpose(y_weights),y_weights))
        # 2.2 Update y_score: the Y latent scores
        y_score = tf.matmul(Y,y_weights)/tf.matmul(tf.transpose(y_weights),y_weights)
        x_weights_diff = tf.math.subtract(x_weights,x_weights_old)
        print(x_weights_diff)
        if tf.matmul(tf.transpose(x_weights_diff),x_weights_diff) < tol or Y.shape[1] ==1:
            break
        if ite == max_iter:
            warnings.warn('Maximum number of iterations reached',
                          ConvergenceWarning)
            break
        x_weights_old = x_weights
        ite += 1
    return x_weights, y_weights, ite

In [117]:
dtype=tf.dtypes.float64

In [118]:
_nipals_tensorflow(Xt,Yt)

(150, 4)


InvalidArgumentError: Incompatible shapes: [4,1] vs. [150,4] [Op:Sub]

In [ ]:
class PLS():
    
    def __init__()